In [1]:
import pandas as pd
import os

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.shape

(200000, 202)

In [4]:
test.shape

(200000, 201)

In [5]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [7]:
targettotrain = train['target']
featurestotrain = train.drop(['ID_code', 'target'], axis=1)
featurestotest = test.drop(['ID_code'], axis=1)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
X_train, X_test, y_train, y_test = train_test_split(featurestotrain, targettotrain, test_size=0.2, random_state=0)

In [10]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
featurestotest = sc.transform(featurestotest)

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.constraints import maxnorm
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
model = Sequential()
model.add(Dense(units=200, input_dim=200, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.005), kernel_constraint=maxnorm(5)))
model.add(Dropout(0.2))
model.add(Dense(units=150, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.005),kernel_constraint=maxnorm(5)))
model.add(Dropout(0.2))
model.add(Dense(units=100, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.005),kernel_constraint=maxnorm(5)))
model.add(Dropout(0.2))
model.add(Dense(units=1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 150)               30150     
_________________________________________________________________
dropout_2 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               15100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total para

In [13]:
# fit model
model.fit(X_train, y_train, batch_size = 16384, epochs = 125, validation_data = (X_test, y_test))

Train on 160000 samples, validate on 40000 samples
Epoch 1/125
160000/160000 [==============================] - 17s 109us/step - loss: 1.5284 - acc: 0.8184 - val_loss: 1.2434 - val_acc: 0.8996
Epoch 2/125
160000/160000 [==============================] - 11s 71us/step - loss: 1.1077 - acc: 0.8995 - val_loss: 0.9835 - val_acc: 0.8996
Epoch 3/125
160000/160000 [==============================] - 12s 76us/step - loss: 0.8964 - acc: 0.8995 - val_loss: 0.7953 - val_acc: 0.8996
Epoch 4/125
160000/160000 [==============================] - 11s 68us/step - loss: 0.7393 - acc: 0.8995 - val_loss: 0.6692 - val_acc: 0.8996
Epoch 5/125
160000/160000 [==============================] - 11s 71us/step - loss: 0.6233 - acc: 0.8995 - val_loss: 0.5719 - val_acc: 0.8996
Epoch 6/125
160000/160000 [==============================] - 12s 74us/step - loss: 0.5373 - acc: 0.8995 - val_loss: 0.4996 - val_acc: 0.8996
Epoch 7/125
160000/160000 [==============================] - 12s 73us/step - loss: 0.4730 - acc: 0.899

160000/160000 [==============================] - 11s 68us/step - loss: 0.2522 - acc: 0.9126 - val_loss: 0.2535 - val_acc: 0.9128
Epoch 59/125
160000/160000 [==============================] - 11s 71us/step - loss: 0.2522 - acc: 0.9126 - val_loss: 0.2537 - val_acc: 0.9124
Epoch 60/125
160000/160000 [==============================] - 11s 69us/step - loss: 0.2523 - acc: 0.9128 - val_loss: 0.2534 - val_acc: 0.9130
Epoch 61/125
160000/160000 [==============================] - 11s 68us/step - loss: 0.2518 - acc: 0.9127 - val_loss: 0.2530 - val_acc: 0.9128
Epoch 62/125
160000/160000 [==============================] - 11s 66us/step - loss: 0.2521 - acc: 0.9128 - val_loss: 0.2532 - val_acc: 0.9128
Epoch 63/125
160000/160000 [==============================] - 11s 68us/step - loss: 0.2519 - acc: 0.9132 - val_loss: 0.2532 - val_acc: 0.9130
Epoch 64/125
160000/160000 [==============================] - 11s 67us/step - loss: 0.2521 - acc: 0.9128 - val_loss: 0.2533 - val_acc: 0.9130
Epoch 65/125
160000

160000/160000 [==============================] - 11s 71us/step - loss: 0.2491 - acc: 0.9139 - val_loss: 0.2500 - val_acc: 0.9134
Epoch 116/125
160000/160000 [==============================] - 10s 62us/step - loss: 0.2491 - acc: 0.9134 - val_loss: 0.2500 - val_acc: 0.9136
Epoch 117/125
160000/160000 [==============================] - 10s 63us/step - loss: 0.2491 - acc: 0.9136 - val_loss: 0.2503 - val_acc: 0.9132
Epoch 118/125
160000/160000 [==============================] - 11s 69us/step - loss: 0.2492 - acc: 0.9140 - val_loss: 0.2505 - val_acc: 0.9137
Epoch 119/125
160000/160000 [==============================] - 13s 82us/step - loss: 0.2495 - acc: 0.9136 - val_loss: 0.2502 - val_acc: 0.9133
Epoch 120/125
160000/160000 [==============================] - 14s 85us/step - loss: 0.2492 - acc: 0.9137 - val_loss: 0.2500 - val_acc: 0.9129
Epoch 121/125
160000/160000 [==============================] - 11s 66us/step - loss: 0.2487 - acc: 0.9136 - val_loss: 0.2499 - val_acc: 0.9134
Epoch 122/125

In [14]:
predicted_y = model.predict(X_test)

In [15]:
print(predicted_y)

[[0.02324648]
 [0.00258362]
 [0.53582466]
 ...
 [0.05960162]
 [0.16806525]
 [0.01064381]]


In [16]:
id_code_test = test['ID_code']

In [17]:
prediction = model.predict(featurestotest)
out = prediction[:,0]

In [18]:
out

array([0.1729722 , 0.21731076, 0.05138029, ..., 0.00252141, 0.05494951,
       0.1330659 ], dtype=float32)